# 라이브러리 로드

In [1]:
import joblib
import psycopg2
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
from tqdm import tqdm

## db 테이블 로드 함수

In [2]:
def find_df(sql):
    # Define connection parameters
    conn = psycopg2.connect(
        host="postgres_db",  # Container name as defined in docker-compose.yml
        database="mydatabase",
        user="myuser",
        password="mypassword"
    )
    # Query data from one of the tables
    # df = pd.read_sql("SELECT * FROM  tn_visit_area_info LIMIT 5;", conn)
    df = pd.read_sql(sql, conn)
    # Close the connection
    df.columns = [x.upper() for x in df.columns]
    conn.close()
    return df
sql = "SELECT * FROM  tn_visit_area_info LIMIT 100;"
df = find_df(sql)

/tmp/ipykernel_174/1441590044.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


## code 데이터 로드

In [3]:
sql ="SELECT * FROM tc_codea"
codea_df = find_df(sql)

sql ="SELECT * FROM tc_codeb"
codeb_df = find_df(sql)

/tmp/ipykernel_174/1441590044.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


# AIHUB 코드 분석

## 1차 전처리

In [4]:
# Get the current timestamp
timestamp = datetime.now()
# visit_E = pd.read_csv('../1.inputdata/tn_visit_area_info_E.csv')
sql = "SELECT * FROM  tn_visit_area_info;"
visit_E = find_df(sql)

id_list=['E']

visit_data_list=[visit_E]

id, visit_area_info = 'E', visit_E
# 관광지 선택
visit_info = visit_area_info[ (visit_area_info['VISIT_AREA_TYPE_CD'] == '1') |
(visit_area_info['VISIT_AREA_TYPE_CD'] == '2') |(visit_area_info['VISIT_AREA_TYPE_CD'] == '3') | (visit_area_info['VISIT_AREA_TYPE_CD'] == '4') |
(visit_area_info['VISIT_AREA_TYPE_CD'] == '5') | (visit_area_info['VISIT_AREA_TYPE_CD'] == '6') |(visit_area_info['VISIT_AREA_TYPE_CD'] == '7') |
 (visit_area_info['VISIT_AREA_TYPE_CD'] == '8')]
visit_info = visit_info.groupby('VISIT_AREA_NM').filter(lambda x: len(x) > 1)
visit_info=visit_info.reset_index(drop = True)

visit_final_E=visit_info

visit_final_E['ratings'] = visit_final_E[['DGSTFN', 'REVISIT_INTENTION', 'RCMDTN_INTENTION']].astype(float).mean(axis=1)

visit_final_E['TRAVELER_ID'] = visit_final_E['TRAVEL_ID'].str.split('_').str[1]


/tmp/ipykernel_174/1441590044.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


- VISIT_AREA_TYPE_CD 기준으로 필터링(1~8)
  > 테이블 내역기준 관광지만 대상으로
- 지역명 존재여부로 필터링
- ratings 생성
  > 만족도(DGSTFN), 재방문의향(REVISIT_INTENTION), 추천의향(RCMDTN_INTENTION) 평균으로 계산  
- TRAVELER_ID 생성

In [5]:
codea_df.query('CD_A in ["VIS", "DGS", "REP", "REC"]')

,IDX,CD_A,CD_NM,CD_MEMO,CD_MEMO2,DEL_FLAG,ORDER_NUM,PERM_WRITE,PERM_EDIT,PERM_DELETE,INS_DT,EDIT_DT
1,6,VIS,방문지유형코드,,,N,999,N,N,N,2022-07-05 10:45:53,None
9,14,DGS,전반적 만족도 점수,,,N,999,N,N,N,2022-07-05 11:55:48,None
10,15,REP,재방문 의향 점수,,,N,999,N,N,N,2022-07-05 11:56:55,None
11,16,REC,추천의향 점수,,,N,999,N,N,N,2022-07-05 11:58:20,None


In [6]:
codeb_df.query('CD_A in ["VIS"]')

,IDX,CD_A,CD_B,CD_NM,CD_MEMO,CD_MEMO2,DEL_FLAG,ORDER_NUM,INS_DT,EDIT_DT
221,1088,VIS,21,집,,,N,999,2022-07-05 15:52:47,NaT
222,1089,VIS,22,친구/친지집,,,N,999,2022-07-05 15:52:56,2022-07-26 16:05:56
223,1090,VIS,23,사무실,,,N,999,2022-07-05 15:53:05,NaT
224,1091,VIS,24,숙소,,,N,999,2022-07-05 15:53:12,NaT
225,1095,VIS,13,체험 활동 관광지,,,N,15,2022-07-14 15:14:57,NaT
226,1002,VIS,1,자연관광지,,,N,10,2022-07-05 10:48:17,NaT
227,1003,VIS,2,"역사/유적/종교 시설(문화재, 박물관, 촬영지, 절 등)",,,N,20,2022-07-05 10:48:24,NaT
228,1005,VIS,3,"문화 시설(공연장, 영화관, 전시관 등)",,,N,30,2022-07-05 11:23:34,NaT
229,1006,VIS,4,"상업지구(거리, 시장, 쇼핑시설)",,,N,40,2022-07-05 11:23:39,NaT
230,1007,VIS,5,"레저/스포츠 관련 시설(스키, 카트, 수상레저)",,,N,50,2022-07-05 11:23:47,NaT


In [7]:
codeb_df.query('CD_A in ["DGS", "REP", "REC"]')

,IDX,CD_A,CD_B,CD_NM,CD_MEMO,CD_MEMO2,DEL_FLAG,ORDER_NUM,INS_DT,EDIT_DT
21,1069,DGS,1,매우 불만족,,,N,999,2022-07-05 11:55:56,NaT
22,1070,DGS,2,불만족,,,N,999,2022-07-05 11:56:01,NaT
23,1071,DGS,3,보통,,,N,999,2022-07-05 11:56:07,NaT
24,1072,DGS,4,만족,,,N,999,2022-07-05 11:56:11,NaT
25,1073,DGS,5,매우만족,,,N,999,2022-07-05 11:56:16,NaT
138,1079,REC,1,전혀 그렇지 않다,,,N,999,2022-07-05 11:58:27,NaT
139,1080,REC,2,그렇지 않다,,,N,999,2022-07-05 11:58:32,NaT
140,1081,REC,3,보통,,,N,999,2022-07-05 11:58:38,NaT
141,1082,REC,4,그렇다,,,N,999,2022-07-05 11:59:30,NaT
142,1083,REC,5,매우 그렇다,,,N,999,2022-07-05 11:59:36,NaT


In [8]:
"""###세부 전처리

####A권역
"""

visit_final_E['SIDO'] = visit_final_E['LOTNO_ADDR'].str.split().str[0]

dfe=visit_final_E
# Group by 'FIRST_WORD' and find the most frequent 'VISIT_AREA_NM' for each group
most_frequent_visits = dfe.groupby('LOTNO_ADDR')['VISIT_AREA_NM'].agg(lambda x: x.mode().iloc[0]).reset_index()

# Merge the most frequent values back to the original DataFrame based on 'FIRST_WORD'
dfe = dfe.merge(most_frequent_visits, on='LOTNO_ADDR', how='left', suffixes=('', '_most_frequent'))

# Update 'VISIT_AREA_NM' with the most frequent values
dfe['VISIT_AREA_NM'] = dfe['VISIT_AREA_NM_most_frequent'].fillna(dfe['VISIT_AREA_NM'])

# Drop temporary columns used for grouping and merging
dfe.drop(columns=['VISIT_AREA_NM_most_frequent'], inplace=True)

dfe[['TRAVELER_ID', 'VISIT_AREA_NM','ratings','SIDO']]

"""####후처리"""

df1 = dfe.rename(columns={'TRAVELER_ID': 'userID','VISIT_AREA_NM': 'itemID','ratings': 'rating'})

df1=df1[['userID','itemID','rating','SIDO']]
# df1.to_csv("../2.preprocessed/dfE.csv")

- 시도명 생성
- 방문지명 최빈값으로 치환
- 유저, 아이템, rating, 시도 만 선택

In [9]:
df1.head()

,userID,itemID,rating,SIDO
0,e000004,화성 관광열차 안내소 연무대 매표소,3.666667,경기
1,e000004,창룡문,4.000000,경기
2,e000004,수원 화성 화홍문,3.333333,경기
3,e000009,농협안성팜랜드,4.000000,경기
4,e000009,스타필드 안성,5.000000,경기


## 모델 학습 테스트

In [10]:
from surprise.dataset import Reader, Dataset
from surprise import SVD

In [11]:
reader = Reader(line_format='user item rating timestamp', sep='\t', rating_scale=(1, 5))
combined_df = pd.concat([df1[['userID', 'itemID', 'rating']], pd.DataFrame([{'userID':"testID", 'itemID':"스타필드 안성", 'rating':3}])]).reset_index(drop=True)
data = Dataset.load_from_df(combined_df, reader)
train_set = data.build_full_trainset()
rcsys = SVD(n_factors=20, n_epochs=20, random_state=9999)
rcsys.fit(train_set)
pred = [rcsys.predict('testID', area_nm) for area_nm in combined_df.itemID.unique() if area_nm not in combined_df.query('userID=="testID"').itemID.values]
pred.sort(key=lambda x: x.est, reverse=True)
pred_df = pd.DataFrame(pred)\
           .assign(est=lambda df: df.est.apply(lambda x: round(x, 1)))

In [12]:
pred_df

,uid,iid,r_ui,est,details
0,testID,KSPO DOME 올림픽 체조경기장,None,4.7,{'was_impossible': False}
1,testID,서울숲,None,4.6,{'was_impossible': False}
2,testID,잠실종합운동장 올림픽 주경기장,None,4.5,{'was_impossible': False}
3,testID,석촌호수 산책길,None,4.5,{'was_impossible': False}
4,testID,에버랜드,None,4.5,{'was_impossible': False}
...,...,...,...,...,...
706,testID,말죽거리,None,3.7,{'was_impossible': False}
707,testID,수원 화성 북동포루,None,3.7,{'was_impossible': False}
708,testID,SETEC,None,3.7,{'was_impossible': False}
709,testID,행궁동 벽화마을,None,3.6,{'was_impossible': False}
